# Step 1: Inital db setup

In [1]:
# SOURCE: https://ndres.me/post/best-jupyter-notebook-extensions/
# autoreload: Autoreloads external files without having to restart the notebook. To enable it:

%load_ext autoreload
%autoreload 2

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
# Initial - Setup environment vars before testing anything
import os
from sqlalchemy import inspect

# import better_exceptions; better_exceptions.hook()

import sys

from IPython.core.debugger import Tracer  # noqa
from IPython.core import ultratb

sys.excepthook = ultratb.FormattedTB(
    mode="Verbose", color_scheme="Linux", call_pdb=True, ostream=sys.__stdout__
)

os.environ["DEBUG"] = "1"
os.environ["TESTING"] = "0"
os.environ["BETTER_EXCEPTIONS"] = "1"

# os.environ["DATABASE_URL"] = "sqlite:///:memory:"
# os.environ["TEST_DATABASE_URL"] = "sqlite:///:memory:"

os.environ["DATABASE_URL"] = "sqlite:///test.db"
os.environ["TEST_DATABASE_URL"] = "sqlite:///test.db"

def debug_dump(obj):
    for attr in dir(obj):
        if hasattr(obj, attr):
            print("obj.%s = %s" % (attr, getattr(obj, attr)))

In [3]:
# Initalize database and create schema etc

import logging

from tenacity import after_log, before_log, retry, stop_after_attempt, wait_fixed

from ultron8.api.db.u_sqlite.session import db_session

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

max_tries = 60 * 5  # 5 minutes
wait_seconds = 1

@retry(
    stop=stop_after_attempt(max_tries),
    wait=wait_fixed(wait_seconds),
    before=before_log(logger, logging.INFO),
    after=after_log(logger, logging.WARN),
)
def init():
    try:
        # Try to create session to check if DB is awake
        db_session.execute("SELECT 1")
    except Exception as e:
        logger.error(e)
        raise e


In [4]:
# Get sqlalchemy classes/objects

from ultron8.api.db.u_sqlite.init_db import init_db
from ultron8.api.db.u_sqlite.session import db_session, engine, Session

# make sure all SQL Alchemy models are imported before initializing DB
# otherwise, SQL Alchemy might fail to initialize properly relationships
# for more details: https://github.com/tiangolo/full-stack-fastapi-postgresql/issues/28
from ultron8.api.db.u_sqlite.base import Base

import pandas as pd

# Import SQLAlchemy data classes

In [5]:
from ultron8.api.db_models.packs import Packs
from ultron8.api.db_models.action import Action

In [6]:
# Tables should be created with Alembic migrations
# But if you don't want to use migrations, create
# the tables un-commenting the next line
# 2 - generate database schema
Base.metadata.create_all(bind=engine)

# 3 - create a new session
session = Session()

2019-07-21 22:30:36,489 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1


INFO:sqlalchemy.engine.base.Engine:SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1


2019-07-21 22:30:36,492 INFO sqlalchemy.engine.base.Engine ()


INFO:sqlalchemy.engine.base.Engine:()


2019-07-21 22:30:36,498 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1


...

INFO:sqlalchemy.engine.base.Engine:CREATE INDEX ix_actions_id ON actions (id)


2019-07-21 22:30:36,777 INFO sqlalchemy.engine.base.Engine ()


INFO:sqlalchemy.engine.base.Engine:()


2019-07-21 22:30:36,783 INFO sqlalchemy.engine.base.Engine COMMIT


INFO:sqlalchemy.engine.base.Engine:COMMIT


In [7]:
# Try initializing everything now
logger.info("Initializing service")
init()
logger.info("Service finished initializing")




INFO:__main__:Initializing service
INFO:__main__:Starting call to '__main__.init', this is the 1st time calling it.


2019-07-21 22:30:36,831 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)


INFO:sqlalchemy.engine.base.Engine:BEGIN (implicit)


2019-07-21 22:30:36,834 INFO sqlalchemy.engine.base.Engine SELECT 1


INFO:sqlalchemy.engine.base.Engine:SELECT 1


2019-07-21 22:30:36,839 INFO sqlalchemy.engine.base.Engine ()


INFO:sqlalchemy.engine.base.Engine:()
INFO:__main__:Service finished initializing


In [8]:
logger.info("Creating initial data")
init_db(db_session)
logger.info("Initial data created")

INFO:__main__:Creating initial data


2019-07-21 22:30:36,899 INFO sqlalchemy.engine.base.Engine SELECT user.id AS user_id, user.full_name AS user_full_name, user.email AS user_email, user.hashed_password AS user_hashed_password, user.is_active AS user_is_active, user.is_superuser AS user_is_superuser 
FROM user 
WHERE user.email = ?
 LIMIT ? OFFSET ?


INFO:sqlalchemy.engine.base.Engine:SELECT user.id AS user_id, user.full_name AS user_full_name, user.email AS user_email, user.hashed_password AS user_hashed_password, user.is_active AS user_is_active, user.is_superuser AS user_is_superuser 
FROM user 
WHERE user.email = ?
 LIMIT ? OFFSET ?


2019-07-21 22:30:36,902 INFO sqlalchemy.engine.base.Engine ('admin', 1, 0)


INFO:sqlalchemy.engine.base.Engine:('admin', 1, 0)


...

INFO:sqlalchemy.engine.base.Engine:BEGIN (implicit)


2019-07-21 22:30:37,266 INFO sqlalchemy.engine.base.Engine SELECT user.id AS user_id, user.full_name AS user_full_name, user.email AS user_email, user.hashed_password AS user_hashed_password, user.is_active AS user_is_active, user.is_superuser AS user_is_superuser 
FROM user 
WHERE user.id = ?


INFO:sqlalchemy.engine.base.Engine:SELECT user.id AS user_id, user.full_name AS user_full_name, user.email AS user_email, user.hashed_password AS user_hashed_password, user.is_active AS user_is_active, user.is_superuser AS user_is_superuser 
FROM user 
WHERE user.id = ?


2019-07-21 22:30:37,269 INFO sqlalchemy.engine.base.Engine (1,)


INFO:sqlalchemy.engine.base.Engine:(1,)
INFO:__main__:Initial data created


In [9]:
debug_dump(Action)

obj.RESOURCE_TYPE = ResourceType.ACTION
obj.UID_FIELDS = ['metadata_file', 'name']
obj.UID_SEPARATOR = :
obj.__class__ = <class 'sqlalchemy.ext.declarative.api.DeclarativeMeta'>
obj.__delattr__ = <slot wrapper '__delattr__' of 'object' objects>
obj.__dict__ = {'__module__': 'ultron8.api.db_models.action', '__doc__': 'Db Schema for Action table.', 'RESOURCE_TYPE': <ResourceType.ACTION: 'action'>, 'UID_FIELDS': ['metadata_file', 'name'], '__tablename__': 'actions', 'id': <sqlalchemy.orm.attributes.InstrumentedAttribute object at 0x10e374bf8>, 'ref': <sqlalchemy.orm.attributes.InstrumentedAttribute object at 0x10e374d58>, 'uid': <sqlalchemy.orm.attributes.InstrumentedAttribute object at 0x10e374e08>, 'metadata_file': <sqlalchemy.orm.attributes.InstrumentedAttribute object at 0x10e374eb8>, 'name': <sqlalchemy.orm.attributes.InstrumentedAttribute object at 0x10e374f68>, 'description': <sqlalchemy.orm.attributes.InstrumentedAttribute object at 0x10e3aa048>, 'runner_type': <sqlalchemy.orm.att

# Step 2: Now That we have a working sqlite session, let's try playing w/ our various db_models classes etc

In [10]:
# Step A. Create a pack ( which has actions, triggers, and sensors associated with it )

# Step 3: Import each of the db_models

In [11]:
# from ultron8.api.db_models.action import Action
# from ultron8.api.db_models.guid import Guid
# from ultron8.api.db_models.item import Item
# from ultron8.api.db_models.packs import Packs
# from ultron8.api.db_models.rule import RuleTypeParameter, RuleType, Rules
# from ultron8.api.db_models.sensors import Sensors
# # from ultron8.api.db_models.timer import TimeDB
# from ultron8.api.db_models.trigger import TriggerType, Trigger, TriggerInstanceDB
# from ultron8.api.db_models.user import User

# Step 4: Ok, let us try inserting, and updating data in the database

# PACKS

In [12]:
##########################################
# packs
##########################################

# p = Packs(name='linux', description='Generic Linux actions', keywords='linux', version='0.1.0', python_versions='3', author='Jarvis', email='info@theblacktonystark.com', contributors='bossjones', files='./tests/fixtures/simple/packs/linux', path="./tests/fixtures/simple/packs/linux", actions=[
#     Action(name="check_loadavg", runner_type="remote-shell-script", description="Check CPU Load Average on a Host", enabled=True, entry_point="checks/check_loadavg.py", parameters='{"period": {"enum": ["1","5","15","all"], "type": "string", "description": "Time period for load avg: 1,5,15 minutes, or \'all\'", "default": "all", "position": 0}}')
# ])

# Create - packs
pack_linux = Packs(
    name="linux",
    description="Generic Linux actions",
    keywords="linux",
    version="0.1.0",
    python_versions="3",
    author="Jarvis",
    email="info@theblacktonystark.com",
    contributors="bossjones",
    files="./tests/fixtures/simple/packs/linux",
    path="./tests/fixtures/simple/packs/linux",
    ref="linux"
)

pack_linux

# inspect pack
# insp = inspect(pack_linux)
# insp

debug_dump(pack_linux)

Packs<name=linux,ref=linux>

obj.RESOURCE_TYPE = ResourceType.PACK
obj.UID_FIELDS = ['ref']
obj.UID_SEPARATOR = :
obj.__class__ = <class 'ultron8.api.db_models.packs.Packs'>
obj.__delattr__ = <method-wrapper '__delattr__' of Packs object at 0x116b377f0>
obj.__dict__ = {'_sa_instance_state': <sqlalchemy.orm.state.InstanceState object at 0x116b37828>, 'name': 'linux', 'description': 'Generic Linux actions', 'keywords': 'linux', 'version': '0.1.0', 'python_versions': '3', 'author': 'Jarvis', 'email': 'info@theblacktonystark.com', 'contributors': 'bossjones', 'files': './tests/fixtures/simple/packs/linux', 'path': './tests/fixtures/simple/packs/linux', 'ref': 'linux', 'uid': 'pack:linux'}
obj.__dir__ = <built-in method __dir__ of Packs object at 0x116b377f0>
obj.__doc__ = Db Schema for Packs table.
obj.__eq__ = <method-wrapper '__eq__' of Packs object at 0x116b377f0>
obj.__format__ = <built-in method __format__ of Packs object at 0x116b377f0>
obj.__ge__ = <method-wrapper '__ge__' of Packs object at 0x116b377f0>
obj.__

In [13]:
# Create - actions

action_check_loadavg = Action(
    name="check_loadavg",
    runner_type="remote-shell-script",
    description="Check CPU Load Average on a Host",
    enabled=True,
    entry_point="checks/check_loadavg.py",
    parameters='{"period": {"enum": ["1","5","15","all"], "type": "string", "description": "Time period for load avg: 1,5,15 minutes, or \'all\'", "default": "all", "position": 0}}',
    pack=pack_linux
)

action_check_loadavg

# action_check_loadavg.dump()

debug_dump(action_check_loadavg)

Action<name=check_loadavg,ref=linux.check_loadavg,runner_type=remote-shell-script,entry_point=checks/check_loadavg.py>

obj.RESOURCE_TYPE = ResourceType.ACTION
obj.UID_FIELDS = ['metadata_file', 'name']
obj.UID_SEPARATOR = :
obj.__class__ = <class 'ultron8.api.db_models.action.Action'>
obj.__delattr__ = <method-wrapper '__delattr__' of Action object at 0x116b37748>
obj.__dict__ = {'_sa_instance_state': <sqlalchemy.orm.state.InstanceState object at 0x116b37710>, 'name': 'check_loadavg', 'runner_type': 'remote-shell-script', 'description': 'Check CPU Load Average on a Host', 'enabled': True, 'entry_point': 'checks/check_loadavg.py', 'parameters': '{"period": {"enum": ["1","5","15","all"], "type": "string", "description": "Time period for load avg: 1,5,15 minutes, or \'all\'", "default": "all", "position": 0}}', 'pack': Packs<name=linux,ref=linux>, 'ref': 'linux.check_loadavg', 'uid': 'action::check_loadavg'}
obj.__dir__ = <built-in method __dir__ of Action object at 0x116b37748>
obj.__doc__ = Db Schema for Action table.
obj.__eq__ = <method-wrapper '__eq__' of Action object at 0x116b37748>
obj.__format__ 

In [14]:
# %debug
# committing pack to db

# session.add(p)

# session.commit()

In [15]:
# p = session.query(Packs).get(1)
# for a in p.actions:
#     print(a, inspect(e).key, a.pack)
# assert set([a.name for e in c.actions]) == set(
#     ["linux"]
# )
# print("\n")

In [16]:
action_check_loadavg

Action<name=check_loadavg,ref=linux.check_loadavg,runner_type=remote-shell-script,entry_point=checks/check_loadavg.py>

In [17]:
# list all variables of global scope
# SOURCE: https://www.dataquest.io/blog/jupyter-notebook-tips-tricks-shortcuts/
%who

Action	 Base	 InteractiveShell	 Packs	 Session	 Tracer	 action_check_loadavg	 after_log	 before_log	 
db_session	 debug_dump	 engine	 init	 init_db	 inspect	 logger	 logging	 max_tries	 
os	 pack_linux	 pd	 retry	 session	 stop_after_attempt	 sys	 ultratb	 wait_fixed	 
wait_seconds	 


In [18]:
# Jupyter-contrib extensions ( MUST HAVE )

# !pip install https://github.com/ipython-contrib/jupyter_contrib_nbextensions/tarball/master
# !pip install jupyter_nbextensions_configurator
# !jupyter contrib nbextension install --user
# !jupyter nbextensions_configurator enable --user

In [19]:
session.commit()